## Teste Logs

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


month = ["jan", "fev", "mar", "abr", "mai", "jun", "jul", "ago", "set", "out", "nov", "dez"]

entradas = 0

for mes in month:
    logs = pd.read_csv(f'logs/in/logs-{mes}.csv', sep=";")
    matriculas = pd.read_csv('data/sei_matriculas.csv')
    logs = logs[logs['target'].str.contains('course')]
    
    print(f"Mes: {mes}, entradas: {logs.shape[0]}, total: {entradas}")
    entradas += int(logs.shape[0])

Mes: jan, entradas: 1365393, total: 0
Mes: fev, entradas: 303221, total: 1365393
Mes: mar, entradas: 921300, total: 1668614
Mes: abr, entradas: 1165973, total: 2589914
Mes: mai, entradas: 1432377, total: 3755887
Mes: jun, entradas: 1147153, total: 5188264
Mes: jul, entradas: 1148609, total: 6335417
Mes: ago, entradas: 2413413, total: 7484026
Mes: set, entradas: 3196248, total: 9897439
Mes: out, entradas: 1968188, total: 13093687
Mes: nov, entradas: 1257663, total: 15061875
Mes: dez, entradas: 844092, total: 16319538


In [1]:
SAVE_OUTPUT = True
ONLY_COURSE_DATA = True
MES = "dez"

### Importo os csv's

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

logs = pd.read_csv(f'logs/in/logs-{MES}.csv', sep=";")
matriculas = pd.read_csv('data/sei_matriculas.csv')

In [3]:
logs

,id_inscricao,id_oferta,id_log,component,action,target,dt_log
0,408987,604,305541537,core,viewed,course,2018-12-01 00:50:50
1,408987,604,305541649,core,updated,course_module_completion,2018-12-01 00:51:11
2,408987,604,305541698,mod_book,viewed,course_module,2018-12-01 00:51:20
3,408987,604,305541699,mod_book,viewed,chapter,2018-12-01 00:51:20
4,408987,604,305541865,core,viewed,course,2018-12-01 00:51:54
...,...,...,...,...,...,...,...
1124778,443465,604,334231018,core,updated,course_module_completion,2019-01-29 23:49:33
1124779,443465,604,334231576,mod_url,viewed,course_module,2019-01-29 23:54:01
1124780,443465,604,334231578,core,updated,course_module_completion,2019-01-29 23:54:01
1124781,443465,604,334231577,core,updated,course_module_completion,2019-01-29 23:54:01


In [4]:
if ONLY_COURSE_DATA:
    logs = logs[logs['target'].str.contains('course')]

### Faço o merge de ambos, inserindo a dt_inscricao no dataset de logs

In [5]:
logs = logs.merge(matriculas, how='left')
logs

,id_inscricao,id_oferta,id_log,component,action,target,dt_log,Unnamed: 0,dt_inscricao
0,408987,604,305541537,core,viewed,course,2018-12-01 00:50:50,63929.0,2018-12-01 00:50:00
1,408987,604,305541649,core,updated,course_module_completion,2018-12-01 00:51:11,63929.0,2018-12-01 00:50:00
2,408987,604,305541698,mod_book,viewed,course_module,2018-12-01 00:51:20,63929.0,2018-12-01 00:50:00
3,408987,604,305541865,core,viewed,course,2018-12-01 00:51:54,63929.0,2018-12-01 00:50:00
4,408987,604,305541908,mod_page,viewed,course_module,2018-12-01 00:52:03,63929.0,2018-12-01 00:50:00
...,...,...,...,...,...,...,...,...,...
844087,443465,604,334231018,core,updated,course_module_completion,2019-01-29 23:49:33,67855.0,2018-12-30 17:27:00
844088,443465,604,334231576,mod_url,viewed,course_module,2019-01-29 23:54:01,67855.0,2018-12-30 17:27:00
844089,443465,604,334231578,core,updated,course_module_completion,2019-01-29 23:54:01,67855.0,2018-12-30 17:27:00
844090,443465,604,334231577,core,updated,course_module_completion,2019-01-29 23:54:01,67855.0,2018-12-30 17:27:00


In [6]:
#logs.dropna(inplace=True)

### Converto todas as colunas de datas para date time, evitando conflitos ao tratar os dados

In [7]:
def convert_columns_to_date_time(dataset, columns_name):
    dataset[[*columns_name]] = dataset[[*columns_name]].fillna(pd.to_datetime("2000-01-01 00:00:00"))
    date_time_columns = pd.to_datetime(dataset[[*columns_name]].stack(), dayfirst=True)
    date_time_columns = date_time_columns.dt.normalize()
    dataset[[*columns_name]] = date_time_columns.unstack()
    return dataset

logs = convert_columns_to_date_time(logs, ['dt_log', 'dt_inscricao'])
logs['dias_log'] = (logs['dt_log'] - logs['dt_inscricao']).dt.days

### Generalizo acessos antes e depois do período [matrícula , 1 mês] para  o valor -1

In [8]:
logs['dias_log'] = logs['dias_log'].clip(lower=-1)
logs.loc[logs['dias_log'] > 30, 'dias_log'] = -1

### Gero um novo dataset que contém apenas id_inscricao e uma nova coluna, dias_log

#### A coluna dias logs contém uma lista com o número referente ao dia em que o aluno acessou o sistema. Repetições ocorrem pelo fato de o aluno acessar e fazer várias ações num mesmo dia.

In [9]:
logs_temp = logs.groupby('id_inscricao')['dias_log'].apply(list).to_frame('dias_log').reset_index()
logs_temp

,id_inscricao,dias_log
0,408987,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,409003,"[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, ..."
2,409007,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
3,409009,[0]
4,409011,"[0, 0, 1, 1, 15, 15, 16, 16, 16, 16, 16, 16, 1..."
...,...,...
4211,443566,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4212,443578,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4213,443587,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -..."
4214,443638,"[0, 0, 0, 0, 0, 0, 0, 0]"


### Insiro novas colunas acesso_dia_n, onde n é um valor que vai de 0 a 30. 

#### Cada coluna representa o acesso do alunos após n dias da sua inscrição, onde o valor 0 significa "não acessou" e 1 "acessou"

In [10]:
def calculate_acess_by_day(dataframe):
    df_dates_between = pd.DataFrame()
    
    for i in range(31):
        df_dates_between[f'acesso_dia_{i}'] = dataframe["dias_log"].apply(lambda s: len(set(s) & set([i])) > 0)
        df_dates_between[f'acesso_dia_{i}']= df_dates_between[f'acesso_dia_{i}'].astype(int)

    return df_dates_between

dif_between_dates = calculate_acess_by_day(logs_temp)

# Por fim, inserimos essas novas colunas no nosso novo dataset:
logs_temp[[*list(dif_between_dates)]] = dif_between_dates

In [11]:
logs_temp = logs_temp.drop("dias_log", axis=1)
logs_temp

,id_inscricao,acesso_dia_0,acesso_dia_1,acesso_dia_2,acesso_dia_3,acesso_dia_4,acesso_dia_5,acesso_dia_6,acesso_dia_7,acesso_dia_8,...,acesso_dia_21,acesso_dia_22,acesso_dia_23,acesso_dia_24,acesso_dia_25,acesso_dia_26,acesso_dia_27,acesso_dia_28,acesso_dia_29,acesso_dia_30
0,408987,1,1,1,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,409003,0,0,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,409007,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,409009,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,409011,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4211,443566,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4212,443578,1,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4213,443587,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4214,443638,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Salvo o novo dataset gerado num outro arquivo

In [12]:
if SAVE_OUTPUT:
    logs_temp.to_csv(f"logs/out/course_only/logs-{MES}.csv")